In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [18]:
housing_train = pd.read_csv('train.csv')
housing_test = pd.read_csv('test.csv')
housing_train["GarageYrBlt"].fillna(0, inplace=True)
housing_train["MasVnrArea"].fillna(0, inplace=True)
#housing_train.info()
X= housing_train[['OverallQual', 'GrLivArea','GarageCars','GarageArea','TotalBsmtSF','1stFlrSF','FullBath','TotRmsAbvGrd','YearBuilt','YearRemodAdd','MasVnrArea','Fireplaces','BsmtFinSF1']]
Y=housing_train['SalePrice'].copy()
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   OverallQual   1460 non-null   int64  
 1   GrLivArea     1460 non-null   int64  
 2   GarageCars    1460 non-null   int64  
 3   GarageArea    1460 non-null   int64  
 4   TotalBsmtSF   1460 non-null   int64  
 5   1stFlrSF      1460 non-null   int64  
 6   FullBath      1460 non-null   int64  
 7   TotRmsAbvGrd  1460 non-null   int64  
 8   YearBuilt     1460 non-null   int64  
 9   YearRemodAdd  1460 non-null   int64  
 10  MasVnrArea    1460 non-null   float64
 11  Fireplaces    1460 non-null   int64  
 12  BsmtFinSF1    1460 non-null   int64  
dtypes: float64(1), int64(12)
memory usage: 148.4 KB


In [3]:
#housing_train.describe()

In [4]:
#%matplotlib inline

In [5]:
#corr_matrix = housing_train.corr()
#corr_matrix["SalePrice"].sort_values(ascending=False)

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
my_pipeline = Pipeline([('imputer', SimpleImputer(strategy="median")),('scaler', StandardScaler())])

In [7]:
housing_num_tr = my_pipeline.fit_transform(X)

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
#model = DecisionTreeRegressor()
model = RandomForestRegressor()
#model = LinearRegression()
model.fit(housing_num_tr,Y)

RandomForestRegressor()

In [9]:
some_data=X.iloc[:100]
some_label=Y.iloc[:100]

In [10]:
prepared_data =my_pipeline.transform(some_data)

In [11]:
model.predict(prepared_data)

array([202834.5 , 173257.37, 218611.78, 150248.1 , 266570.89, 144504.33,
       303932.24, 212292.4 , 148984.  , 119894.5 , 130052.  , 351178.94,
       137732.66, 253852.38, 154342.  , 142652.5 , 148918.5 ,  95020.53,
       156291.5 , 140034.09, 317798.25, 137774.  , 233879.22, 134921.  ,
       148198.75, 246590.23, 132643.  , 303787.66, 189694.5 ,  67889.15,
        63760.  , 144305.  , 191425.52, 164743.  , 284342.26, 305279.44,
       145813.  , 152553.33, 122734.25,  87688.5 , 159398.59, 162999.  ,
       141545.  , 129231.5 , 138235.25, 308667.91, 257103.14, 248370.62,
       113887.85, 127264.83, 175423.8 , 118260.  , 112297.  , 400494.75,
       130317.82, 174403.32, 188156.  , 196087.5 , 425577.58, 124161.66,
       162451.17, 100362.34, 207918.45, 144602.35, 226985.79, 309553.82,
       216434.44, 216767.28,  81893.  , 235899.65, 260327.97, 122693.  ,
       186364.  , 143896.37, 112674.68,  89087.61, 126423.82, 125426.84,
       133198.05, 109980.01, 183821.44, 159918.67, 

In [12]:
some_label

0     208500
1     181500
2     223500
3     140000
4     250000
       ...  
95    185000
96    214000
97     94750
98     83000
99    128950
Name: SalePrice, Length: 100, dtype: int64

In [13]:
from sklearn.metrics import mean_squared_error
import numpy as np
housing_predictions = model.predict(housing_num_tr)
lin_mse = mean_squared_error(Y,housing_predictions)
mse=np.sqrt(lin_mse)

In [14]:
mse

11779.063596625447

In [15]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, housing_num_tr,Y,scoring="neg_mean_squared_error",cv=10)
rsme_scores=np.sqrt(-scores)

In [16]:
rsme_scores

array([28739.45989258, 28418.60923956, 24560.53455609, 46747.9778254 ,
       33372.93858982, 27227.23260532, 23837.97452557, 24892.99125471,
       40673.88875256, 26636.17523674])

In [17]:
print("mean:", rsme_scores.mean())
print("std dev:", rsme_scores.std())

mean: 30510.77824783441
std dev: 7210.6007214256715
